<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###LLama index

In [ ]:
# !pip install llama-index
# !pip install llama-index-llms-azure-openai
# !pip install llama-index-embeddings-azure-openai
# !pip install sqlite-utils
# !pip install python-dotenv

In [3]:
data=[
    {'name':'charls','age':10,'city':'Newyork'},
    {'name':'bob','age':15,'city':'delhi'},
    {'name':'alice','age':20,'city':'mumbai'},
     {'name':'john','age':25,'city':'chicago'},
     {'name':'maddie','age':21,'city':'pune'}
     ]

In [18]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from dotenv import dotenv_values
config = dotenv_values(".env")


azure_openai_key = config["API_KEY"]
azure_openai_endpoint = config["ENDPOINT"]
azure_gpt_model_name=config["GPT_Model_Name"]
gpt_deployment_name=config["Deployment_Name"]
api_version=config['Model_Version']
azure_embed_model_name=config["Embed_Model_Name"]
embedding_deployment_name=config["Embed_Deployment"]

llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

embed_model = AzureOpenAIEmbedding(
    model=azure_embed_model_name,
    deployment_name=embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

In [19]:
from llama_index.core.query_engine import JSONalyzeQueryEngine